In [1]:
include("../src/Dyn3d.jl")
import Dyn3d: ConfigBody, ConfigJoint, SingleBody, SingleJoint, 
              System, Motions, Dof, NumParams, Soln, VertsHistory,
              AddBody, AddJoint, AssembleSystem!, InitSystem!, HERK!, UpdatePosition!

using BenchmarkTools, Compat
using MAT
using Interpolations

In [2]:
# choose config files from folder Config_files
include("../src/Config_files/2dLinkobj.jl")

In [3]:
# add bodys
bodys = Vector{SingleBody}(nbody) # body system
for i = 1:nbody
    bodys[i] = AddBody(i, config_body) # add body
end

# add joints
joints = Vector{SingleJoint}(njoint) # joint system
for i = 1:njoint
    joints[i] = AddJoint(i, config_joints[i]) # add joint
end

# assemble system to a chain
system = System(ndim, nbody, njoint, gravity, num_params)
bodys, joints, system = AssembleSystem!(bodys, joints, system)
system

ndim = 2, njoint = 10, nbody = 10
ndof = 60, nudof = 10, ncdof = 50, np = 9, na = 1
udof = [3, 9, 15, 21, 27, 33, 39, 45, 51, 57]
udof_p = [9, 15, 21, 27, 33, 39, 45, 51, 57]
udof_a = [3]
nudof_HERK = 9, ncdof_HERK = 51
udof_HERK = [9, 15, 21, 27, 33, 39, 45, 51, 57]
gravity = [0.0, 0.0, 0.0]
kinmap = [1 1]


In [4]:
# # test function UpdatePosition!
# bodys, joints, system = UpdatePosition!(bodys, joints, system)

# # test function UpdateVelocity!
# for i = 1:nbody
#     @assert length(bodys[i].v) == 6
#     @assert length(bodys[i].Xp_to_b) == 36
#     @assert length(joints[i].vJ) == 6
# end
# v = ones(Float64,24)
# bodys, joints, system, vJ = UpdateVelocity!(bodys, joints, system, v)

In [5]:
# init system
bodys, joints, system, soln = InitSystem!(bodys, joints, system, scheme)

# init soln structure
solns = (Soln)[]
push!(solns, soln)

# init VertsHistory struct
vs = []
push!(vs, VertsHistory(system.nbody, bodys))

1-element Array{Any,1}:
 [0.0 0.0 0.0707107 0.0707107; 0.0707107 0.0707107 0.141421 0.141421; … ; 0.565685 0.565685 0.636396 0.636396; 0.636396 0.636396 0.707107 0.707107]

[0.0 0.0 0.0707107 0.0707107; 0.0707107 0.0707107 0.141421 0.141421; … ; 0.565685 0.565685 0.636396 0.636396; 0.636396 0.636396 0.707107 0.707107]

[0.0 1.0 1.0 0.0; 0.0 1.0 1.0 0.0; … ; 0.0 1.0 1.0 0.0; 0.0 1.0 1.0 0.0]

In [6]:
# advance in time
idx = 0
@time begin
while soln.t < tf
    # advance one step
    soln, bodys, joints, system = HERK!(soln, bodys, joints, system, scheme, tol)
        
    # record soln and verts_i info
    push!(solns, soln)
    push!(vs, VertsHistory(system.nbody, bodys))
        
    # print progress
    idx += 1
    if mod(idx,500) == 1 @printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt) end
end
@printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
end

itr = 1, t = 0.001, dt = 3.024582e-05
itr = 501, t = 0.023, dt = 5.755930e-05
itr = 1001, t = 0.052, dt = 5.757103e-05
itr = 1501, t = 0.081, dt = 5.758328e-05
itr = 2001, t = 0.110, dt = 5.760865e-05
itr = 2501, t = 0.139, dt = 5.765330e-05
itr = 3001, t = 0.168, dt = 5.771940e-05
itr = 3501, t = 0.196, dt = 5.780626e-05
itr = 4001, t = 0.225, dt = 5.791119e-05
itr = 4501, t = 0.254, dt = 5.802944e-05
itr = 5001, t = 0.283, dt = 5.815385e-05
itr = 5501, t = 0.312, dt = 5.827500e-05
itr = 6001, t = 0.342, dt = 5.838199e-05
itr = 6501, t = 0.371, dt = 5.846371e-05
itr = 7001, t = 0.400, dt = 5.851034e-05
itr = 7501, t = 0.429, dt = 5.851479e-05
itr = 8001, t = 0.459, dt = 5.847393e-05
itr = 8501, t = 0.488, dt = 5.838909e-05
itr = 9001, t = 0.517, dt = 5.826605e-05
itr = 9501, t = 0.546, dt = 5.811478e-05
itr = 10001, t = 0.575, dt = 5.794901e-05
itr = 10501, t = 0.604, dt = 5.778652e-05
itr = 11001, t = 0.633, dt = 5.765152e-05
itr = 11501, t = 0.662, dt = 5.757727e-05
itr = 12001, t =

In [7]:
# create regular time grid and acquire solutions on it
qJ_regs = Float64[]
t_reg = linspace(0,solns[end].t,length(solns))
for i = 1:system.ndof
    t_temp = ([solns[k].t for k = 1:length(solns)],)
    qJ_temp = [solns[k].qJ[i] for k = 1:length(solns)]   
    qJ_reg = interpolate(t_temp, qJ_temp, Gridded(Linear()))[t_reg]
    append!(qJ_regs, qJ_reg)
end
qJ_regs = reshape(qJ_regs,(length(solns), system.ndof))

# get verts info based on this regular grid solution
vs_reg = []
bodys_reg = deepcopy(bodys)
joints_reg = deepcopy(joints)
system_reg = deepcopy(system)
for i = 1:length(solns)
    bodys_reg, joints_reg, system_reg = UpdatePosition!(bodys_reg, joints_reg, system_reg, solns[i].qJ)
    push!(vs_reg, VertsHistory(system.nbody, bodys_reg))
end

# write to .mat file for animation
matwrite("../matlab_plot/verts_i.mat", Dict(
    "ndim" => system.ndim,
    "nbody" => system.nbody,
    "nverts" => bodys[1].nverts,
    "t" => collect(t_reg),
    "verts" => vs_reg
))

In [8]:
# # results printed
# solns[end].qJ
# solns[end].v
# solns[end].v̇
# solns[end].λ
# solns[end].t

# # code analysis
# @benchmark HERK!(soln, bodys, joints, system, scheme, tol)
# @profile HERK!(soln, bodys, joints, system, scheme, tol)
# Profile.print()

6 ./task.jl:335; (::IJulia.##14#17)()
 6 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  6 ...6/Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1},...
   6 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::I...
    6 ./loading.jl:522; include_string(::String, ::String)
     6 ./<missing>:?; anonymous
      6 ./profile.jl:23; macro expansion
       2 .../src/TimeMarching.jl:126; HERK!(::Dyn3d.ConstructSystem....
        1 ...src/TimeMarching.jl:212; HERKFuncf(::Array{Dyn3d.Constr...
         1 ./array.jl:1164; vcat(::Array{Float64,1}, ::Ar...
        1 ...src/TimeMarching.jl:250; HERKFuncf(::Array{Dyn3d.Constr...
         1 ./operators.jl:424; *
          1 ./linalg/matmul.jl:483; generic_matmatmul!(::Array{Fl...
           1 ./linalg/matmul.jl:548; _generic_matmatmul!(::Array{...
       2 .../src/TimeMarching.jl:138; HERK!(::Dyn3d.ConstructSystem....
        2 ...src/TimeMarching.jl:295; HERKFuncG(::Array{Dyn3d.Constr...
         2 ./linalg/matmul.jl:483; ge